# Install

In [1]:
%%capture
!pip install -U datasets pandas numpy sklearn

In [2]:
%%capture
!pip install --no-cache-dir transformers sentencepiece

# Utility

In [3]:
import requests
from pathlib import Path
from tqdm.auto import tqdm

def download_from_url(url: str) -> str:
    request = requests.get(
        url, 
        allow_redirects=True, 
        stream=True)
    total_size_in_bytes= int(request.headers.get('content-length', 0))
    block_size = 2**10 # 1 Kibibyte

    filename = url.split("/")[-1]

    with tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True) as progress_bar:
        with open(filename, 'wb') as f:
            for data in request.iter_content(block_size):
                progress_bar.update(len(data))
                f.write(data)

    return filename

In [4]:
import tarfile
import zipfile

def unpack_download(filename: str) -> None:
    if ".tar.gz" in filename:
        with tarfile.open(filename, 'r:gz') as tar_ref:
            for file in tqdm(iterable=tar_ref.getmembers(), total=len(tar_ref.getmembers())):
                tar_ref.extract(member=file)
    elif ".zip" in filename:
        with zipfile.ZipFile(filename, 'r') as zip_ref:
            for file in tqdm(iterable=zip_ref.namelist(), total=len(zip_ref.namelist())):
                zip_ref.extract(member=file)
    else:
        raise ValueError(f"Unknown file extension '{filename}'.")

# Download datasets

In [5]:
germanquad_download_link = "https://germanquad.s3.amazonaws.com/GermanQuAD.zip" # From https://www.deepset.ai/germanquad
squad_v1_1_download_link = "https://data.deepai.org/squad1.1.zip" # From https://deepai.org/dataset/squad
xquad_download_link = "https://github.com/deepmind/xquad/archive/refs/heads/master.zip" # From https://github.com/deepmind/xquad
mlqa_download_link = "https://dl.fbaipublicfiles.com/MLQA/MLQA_V1.zip" # From https://github.com/facebookresearch/MLQA
machine_translated_squad_train_link = "https://dl.fbaipublicfiles.com/MLQA/mlqa-translate-train.tar.gz" # From https://github.com/facebookresearch/MLQA

dataset_links = [
    germanquad_download_link,
    squad_v1_1_download_link,
    xquad_download_link,
    mlqa_download_link,
    machine_translated_squad_train_link,
]

In [6]:
for link in dataset_links:
    print("Downloading", link)
    filename = download_from_url(link)
    print("Unpacking", filename)
    unpack_download(filename)

  0%|          | 0.00/2.73M [00:00<?, ?iB/s]

Unpacking GermanQuAD.zip


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0.00/9.15M [00:00<?, ?iB/s]

Unpacking squad1.1.zip


...

Unpacking MLQA_V1.zip


  0%|          | 0/102 [00:00<?, ?it/s]

  0%|          | 0.00/63.4M [00:00<?, ?iB/s]

Unpacking mlqa-translate-train.tar.gz


  0%|          | 0/16 [00:00<?, ?it/s]

# Prepare datasets for model training

Download locations:

| Datensatz | Originale Aufteilung | QA-Paare Training | QA-Paare Test / Validierung | QA-Paare Evaluation | Anzahl QA-Paar Gesamt | Quelle |
|---|---|---|---|---|---|---|
| SQuAD | Training, Test | 87.599 (89 \%) | 10.570 (11\%) | - | 98.169 | https://deepai.org/dataset/squad |
| GermanQuAD | Training, Test | 11.518 (84 \%) | 2204 (16 \%) | - | 13.722 | https://www.deepset.ai/germanquad |
| Maschinell übersetztes SQuAD (MLQA) | Training, Test | 80.069 (89 \%) | 9.927 (11 \%) | - | 89.996 | https://github.com/facebookresearch/MLQA#translate-train-and-translate-test-data |
| MLQA | Test, Evaluation | - | 512 | 4.517 | 5.029 | https://github.com/facebookresearch/MLQA |
| XQuAD | Evaluation | - | - | 1.190 | 1.190 | https://github.com/deepmind/xquad |


In [7]:
from pathlib import Path

# GermanQuAD
germanquad_path = Path("GermanQuAD")
germanquad_train_path = germanquad_path / "GermanQuAD_train.json"
germanquad_test_path = germanquad_path / "GermanQuAD_test.json"

# SQuAD (English)
squad_train_path = Path("train-v1.1.json")
squad_test_path = Path("dev-v1.1.json")

# XQuAD
xquad_path = Path("xquad-master")
xquad_de_path = xquad_path / "xquad.de.json"

# MLQA
mlqa_path = Path("MLQA_V1")
mlqa_dev_path = mlqa_path / "dev" / "dev-context-de-question-de.json"
mlqa_test_path = mlqa_path / "test" / "test-context-de-question-de.json"

# SQuAD Machine Translated German (from MLQA Paper / Repository)
squad_machine_translated_path = Path("mlqa-translate-train")
squad_machine_translated_train_path = (squad_machine_translated_path /
                                  "de_squad-translate-train-train-v1.1.json")
squad_machine_translated_test_path = (squad_machine_translated_path /
                                  "de_squad-translate-train-dev-v1.1.json")

In [8]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("google/mt5-base", use_fast=False)
tokenizer.add_tokens(["<sep>", "<hl>"])

Downloading:   0%|          | 0.00/376 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/702 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

2

In [9]:
def preprocess_function(samples):
    inputs = []
    targets = []

    for i in range(len(samples["input_text"])):
        input = samples["prefix"][i] + " " + samples["input_text"][i] + tokenizer.eos_token
        inputs.append(input)
        target = samples["target_text"][i] + tokenizer.eos_token
        targets.append(samples["target_text"][i] + tokenizer.eos_token)
  
    model_inputs = tokenizer(
        inputs,
        max_length=max_input_len,
        padding='do_not_pad',
        truncation=False,
    )
    
    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets, 
            max_length=max_label_len, 
            padding='do_not_pad', 
            truncation=False
        )

    labels["input_ids"] = [
        [(l if l != tokenizer.pad_token_id else -100) for l in label]
        for label in labels["input_ids"]
    ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
import json

qg_type = "e2e" #@param ["e2e", "answer-aware"]

def generate_examples_squad(filepath, prefix = "generate question: "):
    """This function returns the examples in the raw (text) form."""
    def highlight_text(text: str, 
                        context: str, 
                        highlight_token: str = "<hl>") -> str:
        return context.replace(text, f"{highlight_token}{text}{highlight_token}")

    with open(filepath, encoding="utf-8") as f:
        squad = json.load(f)
        if qg_type == "e2e":
          for article in squad["data"]:
              title = article.get("title", "")
              for paragraph in article["paragraphs"]:
                  context = paragraph["context"].split("==\n", 1)[-1].replace("\n", " ").replace("''", "'")  # do not strip leading blank spaces GH-2585
                  context = context + " " + tokenizer.eos_token
                  prefix_tokens = tokenizer(
                      prefix,
                      padding='do_not_pad',
                      truncation=False,
                  )['input_ids']
                  context_tokens = tokenizer(
                      context,
                      padding='do_not_pad',
                      truncation=False,
                  )['input_ids']
                  questions = [qa["question"].strip() for qa in paragraph["qas"]]
                  questions = " <sep> ".join(questions)
                  questions += " " + tokenizer.eos_token
                  with tokenizer.as_target_tokenizer():
                      label_tokens = tokenizer(
                          questions, 
                          padding='do_not_pad', 
                          truncation=False
                      )['input_ids']

                      yield {
                          "prefix": prefix,
                          "input_text": context,
                          "input_token_len": len(prefix_tokens) + len(context_tokens),
                          "target_text": questions,
                          "label_token_len": len(label_tokens)
                      }
        elif qg_type == "answer-aware":
            for article in squad["data"]:
              title = article.get("title", "")
              for paragraph in article["paragraphs"]:
                  context = paragraph["context"].split("==\n", 1)[-1].replace("\n", " ").replace("''", "'")  # do not strip leading blank spaces GH-2585
                  for qa in paragraph["qas"]:
                      question = qa["question"] + tokenizer.eos_token
                      answer = qa["answers"][0]["text"]
                      highlighted_context = highlight_text(answer, context) + tokenizer.eos_token
                      prefix_tokens = tokenizer(
                          prefix,
                          padding='do_not_pad',
                          truncation=False,
                      )['input_ids']
                      context_tokens = tokenizer(
                          highlighted_context,
                          padding='do_not_pad',
                          truncation=False,
                      )['input_ids']
                      with tokenizer.as_target_tokenizer():
                          label_tokens = tokenizer(
                              question,
                              padding='do_not_pad',
                              truncation=False
                          )['input_ids']

                      yield {
                          "prefix": prefix, 
                          "input_text": highlighted_context,
                          "input_token_len": len(prefix_tokens) + len(context_tokens),
                          "target_text": question,
                          "label_token_len": len(label_tokens)
                      }


## Combination of datasets

In [11]:
import pandas as pd
import numpy as np

# merge squad
squad_train = pd.DataFrame(list(generate_examples_squad(str(squad_train_path.resolve()))))
squad_dev = pd.DataFrame(list(generate_examples_squad(str(squad_test_path.resolve()))))

# merge translated squad
squad_translated_train = pd.DataFrame(list(generate_examples_squad(str(squad_machine_translated_train_path.resolve()))))
squad_translated_dev = pd.DataFrame(list(generate_examples_squad(str(squad_machine_translated_test_path.resolve()))))

# merge germanquad
germanquad_train = pd.DataFrame(list(generate_examples_squad(str(germanquad_train_path.resolve()))))
germanquad_test = pd.DataFrame(list(generate_examples_squad(str(germanquad_test_path.resolve()))))

# xquad (eval dataset)
xquad_eval = pd.DataFrame(list(generate_examples_squad(str(xquad_de_path.resolve())))) # Überschneidung mit SQuAD Test

# mlqa (isn't used) -> Too little average questions per context for answer-agnostic QG (approx. 1) 
# SQuAD and GermanQuAD have approx. 5 questions per context
mlqa_dev = pd.DataFrame(list(generate_examples_squad(str(mlqa_dev_path.resolve()))))
mlqa_test = pd.DataFrame(list(generate_examples_squad(str(mlqa_test_path.resolve()))))
mlqa_full = pd.concat([mlqa_dev, mlqa_test], ignore_index=True)

/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:220: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated"


## ToDo: Change used datasets here

In [56]:
# combinate datasets for generation
# The index has to be ignored because otherwise too many items will be deleted later on (pandas specific)
dataset_train = pd.concat([germanquad_train], ignore_index=True)
dataset_test = pd.concat([germanquad_test], ignore_index=True)

# Filter dataset

In [57]:
max_input_len = 1024 #@param {type:"integer"}
max_label_len = 128 #@param {type:"integer"}

def filter_examples(df, max_input_length: int = max_input_len, max_label_length: int = max_label_len):
    df.drop(
        df[
           (df.input_token_len > max_input_length) | 
           (df.label_token_len > max_label_length)].index, 
        inplace=True)

In [58]:
from google.colab import data_table
#data_table.DataTable(dataset_train, include_index=False, num_rows_per_page=5)
dataset_train

,prefix,input_text,input_token_len,target_text,label_token_len
0,generate question:,Obwohl die Vereinigten Staaten wie auch viele ...,461,Von welchem Gesetzt stammt das Amerikanische a...,30
1,generate question:,Die theologische Anthropologie als Teilbereich...,129,Wie viele ethnische Gruppen und indigenen Völk...,17
2,generate question:,Bis jetzt sind noch keine endgültigen und gena...,279,Woher kommt die Sexuelle Orientierung von Mens...,14
3,generate question:,Seit 1946 steht das Territorium auf der UN-Lis...,159,Seit wann gehört Guam zu dem Gebiet der Verein...,37
4,generate question:,Lange Zeit konnte man bei der EWG und EG von e...,201,Was ist die EU laut Bundesverfassungsgericht? ...,24
...,...,...,...,...,...
39904,generate question:,"Das Institut für Medizin, das zentrale College...",133,Von welcher Universität ist das Institut für M...,72
39905,generate question:,Fußball und Cricket sind die beliebtesten Spor...,261,"Zusammen mit Cricket, welcher Sport ist bei de...",89
39906,generate question:,Die Gesamtlänge der Straßen in Nepal wird ab 2...,157,Warum ist Reisen in Kathmandu hauptsächlich üb...,90
39907,generate question:,"Der wichtigste internationale Flughafen, der K...",248,Was ist der primäre Flughafen Nepals für inter...,64


In [59]:
#data_table.DataTable(dataset_test, include_index=False, num_rows_per_page=5)
dataset_test

,prefix,input_text,input_token_len,target_text,label_token_len
0,generate question:,An der RWTH Aachen im Institut für Elektrische...,424,Was kann den Verschleiß des seillosen Aufzuges...,75
1,generate question:,"Sichuan gilt vorwiegend als Reisanbaugebiet, i...",198,Was wird in Sichuan angebaut? <sep> Welches fü...,54
2,generate question:,Antennen strahlen polarisierte Wellen ab. Als ...,324,"Was passiert, wenn die Polarisation einer Empf...",60
3,generate question:,'Globales Ziel' des Softwaretestens ist das 'M...,335,Was ist das Hauptziel von Softwaretest? <sep> ...,24
4,generate question:,Aufgrund der Vielzahl ihrer Symptome wurde die...,1151,Warum konnte man die Tuberkulose bis zum 19. J...,230
...,...,...,...,...,...
4556,generate question:,"Wo ist die Masse des Objekts, ist die Geschwin...",257,Wo geht Zentripetalkraft Force hin? <sep> Wie ...,86
4557,generate question:,"Eine konservative Kraft, die auf einem geschlo...",175,"Was ist die einzige Form, in die potenzielle E...",101
4558,generate question:,Für bestimmte körperliche Szenarien ist es unm...,191,Was ist manchmal unmöglich zu modellieren? <se...,56
4559,generate question:,Die Verbindung zwischen makroskopischen noncon...,134,In welcher Behandlung werden nonconservative u...,82


In [60]:
filter_examples(dataset_train)
filter_examples(dataset_test)

In [61]:
from google.colab import data_table
#data_table.DataTable(dataset_train, include_index=False, num_rows_per_page=5)
dataset_train

,prefix,input_text,input_token_len,target_text,label_token_len
0,generate question:,Obwohl die Vereinigten Staaten wie auch viele ...,461,Von welchem Gesetzt stammt das Amerikanische a...,30
1,generate question:,Die theologische Anthropologie als Teilbereich...,129,Wie viele ethnische Gruppen und indigenen Völk...,17
2,generate question:,Bis jetzt sind noch keine endgültigen und gena...,279,Woher kommt die Sexuelle Orientierung von Mens...,14
3,generate question:,Seit 1946 steht das Territorium auf der UN-Lis...,159,Seit wann gehört Guam zu dem Gebiet der Verein...,37
4,generate question:,Lange Zeit konnte man bei der EWG und EG von e...,201,Was ist die EU laut Bundesverfassungsgericht? ...,24
...,...,...,...,...,...
39904,generate question:,"Das Institut für Medizin, das zentrale College...",133,Von welcher Universität ist das Institut für M...,72
39905,generate question:,Fußball und Cricket sind die beliebtesten Spor...,261,"Zusammen mit Cricket, welcher Sport ist bei de...",89
39906,generate question:,Die Gesamtlänge der Straßen in Nepal wird ab 2...,157,Warum ist Reisen in Kathmandu hauptsächlich üb...,90
39907,generate question:,"Der wichtigste internationale Flughafen, der K...",248,Was ist der primäre Flughafen Nepals für inter...,64


In [62]:
#data_table.DataTable(dataset_test, include_index=False, num_rows_per_page=5)
dataset_test

,prefix,input_text,input_token_len,target_text,label_token_len
0,generate question:,An der RWTH Aachen im Institut für Elektrische...,424,Was kann den Verschleiß des seillosen Aufzuges...,75
1,generate question:,"Sichuan gilt vorwiegend als Reisanbaugebiet, i...",198,Was wird in Sichuan angebaut? <sep> Welches fü...,54
2,generate question:,Antennen strahlen polarisierte Wellen ab. Als ...,324,"Was passiert, wenn die Polarisation einer Empf...",60
3,generate question:,'Globales Ziel' des Softwaretestens ist das 'M...,335,Was ist das Hauptziel von Softwaretest? <sep> ...,24
5,generate question:,"Als Barcelona 1899 gegründet wurde, gab es noc...",446,Wo spielte der FC Barcelona bei dessen Gründun...,82
...,...,...,...,...,...
4556,generate question:,"Wo ist die Masse des Objekts, ist die Geschwin...",257,Wo geht Zentripetalkraft Force hin? <sep> Wie ...,86
4557,generate question:,"Eine konservative Kraft, die auf einem geschlo...",175,"Was ist die einzige Form, in die potenzielle E...",101
4558,generate question:,Für bestimmte körperliche Szenarien ist es unm...,191,Was ist manchmal unmöglich zu modellieren? <se...,56
4559,generate question:,Die Verbindung zwischen makroskopischen noncon...,134,In welcher Behandlung werden nonconservative u...,82


# Save to disk

In [63]:
from pathlib import Path
from datasets import DatasetDict, Dataset, load_from_disk

splits = {}
splits["train"] = Dataset.from_pandas(dataset_train.reset_index(drop=True).drop(['input_token_len', 'label_token_len'], axis=1))
splits["validation"] = Dataset.from_pandas(dataset_test.reset_index(drop=True).drop(['input_token_len', 'label_token_len'], axis=1))
# if test is not None:
#     splits["test"] = Dataset.from_pandas(test.reset_index(drop=True).drop(['input_token_len', 'label_token_len'], axis=1))

datasetdict = DatasetDict(splits)
suffix = "-qg-germanquad" #@param {type:"string"}
path_string = str(Path() / f"{qg_type}{suffix}")
datasetdict.save_to_disk(path_string)


In [64]:
import shutil
zip_file = shutil.make_archive(path_string, 'zip', path_string)

In [65]:
from google.colab import files
files.download(zip_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Load test

In [66]:
from datasets import load_from_disk
loaded_datasetdict = load_from_disk(path_string)
print(loaded_datasetdict['train'][0])
loaded_datasetdict

{'prefix': 'generate question: ', 'input_text': "Obwohl die Vereinigten Staaten wie auch viele Staaten des Commonwealth Erben des britischen Common Laws sind, setzt sich das amerikanische Recht bedeutend davon ab. Dies rührt größtenteils von dem langen Zeitraum her, in dem sich das amerikanische Recht unabhängig vom Britischen entwickelt hat. Entsprechend schauen die Gerichte in den Vereinigten Staaten bei der Analyse von eventuell zutreffenden britischen Rechtsprinzipien im Common Law gewöhnlich nur bis ins frühe 19. Jahrhundert. Während es in den Commonwealth-Staaten üblich ist, dass Gerichte sich Entscheidungen und Prinzipien aus anderen Commonwealth-Staaten importieren, ist das in der amerikanischen Rechtsprechung selten. Ausnahmen bestehen hier nur, wenn sich überhaupt keine relevanten amerikanischen Fälle finden lassen, die Fakten nahezu identisch sind und die Begründung außerordentlich überzeugend ist. Frühe amerikanische Entscheidungen zitierten oft britische Fälle, solche Zita

DatasetDict({
    train: Dataset({
        features: ['prefix', 'input_text', 'target_text'],
        num_rows: 37432
    })
    validation: Dataset({
        features: ['prefix', 'input_text', 'target_text'],
        num_rows: 4120
    })
})

input length: 1024

label length: 128

Trainings examples answer-agnostic question generation:

| Dataset | Train | Train (filtered) | Train filtered diff | Test | Test (filtered) | Test filtered diff |
|---|---|---|---|---|---|---|
| SQuAD | 18896 | 17964 | 932 | 2067 | 1907 | 160 |
| GermanQuAD | 2540 | 2288 | 252 | 474 | 395 | 79 |
| Translated SQuAD (MLQA) | 18473 | 17180 | 1293 | 2020 | 1818 | 202 |
| SQuAD, GermanQuAD | 21436 | 20252 | 1184 | 2541 | 2302 | 239 |
| GermanQuAD, Übersetztes SQuAD | 21013 | 19468 | 1545 | 2494 | 2213 | 281 |
| SQuAD, GermanQuAD, Übersetztes SQuAD | 39909 | 37432 | 2477 | 4561 | 4120 |  41 | 
| MLQA | 4053 | 4019 | 34 | 456 | 453 | 3 |
| XQuAD | 240 | 212 | 28 | - | - | - |